Run this file locally. Collab doesn't support asyncpraw properly which can cause 401 issues during scrapping.

* Here negative means negative examples, positive means positive examples and neutral means neutral examples

* Uncomment the 'negative' subreddits and run all the files and save the 'negative' dataset.
Then do this for postive and neutral too

* Finally combine all these 3 datasets manually via excel or similar tool and save the file as `Reddit-data-combined`

In [1]:
!pip install asyncpraw nest_asyncio numpy pandas openpyxl --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 7.3 MB/s eta 0:00:00


In [2]:
import asyncio
import asyncpraw
import nest_asyncio
import pandas as pd
import re
import time
from datetime import datetime
import numpy as np

In [3]:
# Negative Examples
subreddits = [
    "depression", "SuicideWatch", "selfharm", "selfharmteens", "mentalhealth","SelfHate", "SuicideBereavement",
    "bullying", "Anxiety", "BPD", "CPTSD", "depression_help", "mentalillness", "confidence", "needadvice",
    "offmychest", "PanicAttack", "PTSD", "sad", "socialanxiety", "MentalHealthUK", "emotionalneglect",
    "SuicidePrevention", "traumatoolbox", "twosentencehorror",
    "venting", "bipolar", "eating_disorders", "foreveralone", "lonely",
    "mentalhealthsupport", "OCD", "psychoticreddit", "schizophrenia", "selfhelp",
    "stress", "Therapy", "trauma", "TrueOffMyChest", "UnsentLetters", "addiction",
    "alcoholism", "getting_over_it", "griefsupport", "hardshipmates",
    "helpme", "mentalhealthmatters", "needafriend", "runaway", "teen_venting"
]

In [ ]:
# # POSITIVE
# subreddits = [
#     "GoodNews", "GetMotivated", "MadeMeSmile", "HumansBeingBros", "Happy", "DecidingToBeBetter",
#     " KindVoice", "RandomActsOfKindness", "positivityDaily", "GetMotivated", "happydayswasgreat","GetMotivatedBuddies"
# ]

In [ ]:
# # Neutral
# subreddits = [
#     "NeutralPolitics", "AskHistorians", "ExplainLikeImFive", "AskScience", "TrueAskReddit", "DepthHub",
#     "ChangeMyView", "OutOfTheLoop", "TodayILearned", "WritingPrompts","LegalAdvice", "AskScienceFiction",
#     "AskOuija", "PointlessStories", "WhatIsThisThing"
# ]

In [4]:
len(subreddits)

50

### Scrapping Reddit

In [5]:
from getpass import getpass
ID = getpass('Enter client_id: ')

Enter client_id: ··········


In [6]:
SECRET = getpass('Enter client_secret: ')

Enter client_secret: ··········


In [7]:
reddit = asyncpraw.Reddit(client_id=ID,
                          client_secret=SECRET,
                          user_agent='Scraper 1.0 by /u/HimGoneWild')

In [8]:
async def fetch_posts():
    data = []
    for subreddit_name in subreddits:
        try:
            subreddit = await reddit.subreddit(subreddit_name)
            async for submission in subreddit.hot(limit=250):
                # Extract the first paragraph or first few sentences
                first_paragraph = submission.selftext.strip().split('\n')[0]

                # Extract sentences containing specific keywords
                keywords = ['depression', 'suicide', 'bullying', 'stress', 'anxiety','kill','live','life','harm']
                relevant_sentences = []
                for sentence in re.split(r'\. |\n', submission.selftext.strip()):
                    if any(keyword in sentence.lower() for keyword in keywords):
                        relevant_sentences.append(sentence.strip())

                selective_content = " ".join(relevant_sentences) if relevant_sentences else first_paragraph

                # Extract additional metadata
                post_time = datetime.fromtimestamp(submission.created_utc).isoformat()
                data.append({
                    "title": submission.title,
                    "selective_selftext": selective_content,
                    "subreddit": subreddit_name,
                    "post_time": post_time,
                    "upvotes": submission.score,
                    "num_comments": submission.num_comments
                })
            # Delay to handle rate limiting
            time.sleep(2)
        except Exception as e:
            print(f"Error fetching data from {subreddit_name}: {e}")
    return data

In [9]:
# Run the asynchronous function and store the results in a DataFrame
data = await fetch_posts()

Error fetching data from selfharmteens: received 404 HTTP response


In [ ]:
df = pd.DataFrame(data)
df.head()

""


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12096 entries, 0 to 12095
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               12096 non-null  object
 1   selective_selftext  12096 non-null  object
 2   subreddit           12096 non-null  object
 3   post_time           12096 non-null  object
 4   upvotes             12096 non-null  int64 
 5   num_comments        12096 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 567.1+ KB


In [ ]:
# Remove rows where 'selective_selftext' is an empty string
df = df[df['selective_selftext'] != '']

# .replace() method to convert empty strings to NaN and then drop them
df['selective_selftext'] = df['selective_selftext'].replace('', np.nan)
df.dropna(subset=['selective_selftext'], inplace=True)

In [ ]:
df = df[~df['selective_selftext'].str.contains(r'http[s]?://', na=False)]

In [ ]:
df.head()

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.info()

In [ ]:
df = df[df['selective_selftext'].apply(lambda x: len(x.split()) > 5)]

In [ ]:
df.info()

In [ ]:
df.value_counts("subreddit")

In [ ]:
df.rename(columns={'selective_selftext': 'post_text'}, inplace=True)

In [ ]:
df.head()

In [ ]:
# df.to_excel('positive-dataset.xlsx', index=False)
# df.to_excel('positive-dataset.xlsx', index=False)
# df.to_excel('neutral-dataset.xlsx', index=False)